In [46]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from PIL import Image

In [16]:
train_ds, val_ds = image_dataset_from_directory(
    '../Data/proccessed',
    batch_size=32,
    image_size=(224,224),
    subset='both',
    seed=18,
    validation_split=0.2,
)

Found 2671 files belonging to 4 classes.
Using 2137 files for training.
Using 534 files for validation.


In [12]:
base = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)

base.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [17]:
inputs = Input(shape=(224,224,3))

x = preprocess_input(inputs)

In [18]:
x = base(x)

In [21]:
x = GlobalAveragePooling2D()(x)

In [22]:
x = Dense(128, activation='relu')(x)

In [25]:
outputs = Dense(4, activation='softmax')(x)

In [26]:
model = Model(inputs=inputs, outputs = outputs)

In [27]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [29]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3, 
    restore_best_weights=True
)

In [32]:
checkpoint = ModelCheckpoint('model_ckeckpoint.weights.h5',
                             monitor='val_accuracy',
                             save_weights_only=True
)

In [33]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=10, 
          callbacks=[early_stop,checkpoint]
)

Epoch 1/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 31s 402ms/step - accuracy: 0.7178 - loss: 0.6561 - val_accuracy: 0.8034 - val_loss: 0.4791
Epoch 2/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 28s 413ms/step - accuracy: 0.8652 - loss: 0.3315 - val_accuracy: 0.8315 - val_loss: 0.3870
Epoch 3/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 26s 381ms/step - accuracy: 0.9106 - loss: 0.2406 - val_accuracy: 0.8240 - val_loss: 0.3957
Epoch 4/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 25s 380ms/step - accuracy: 0.9209 - loss: 0.2011 - val_accuracy: 0.8446 - val_loss: 0.3561
Epoch 5/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 25s 380ms/step - accuracy: 0.9565 - loss: 0.1401 - val_accuracy: 0.8764 - val_loss: 0.3536
Epoch 6/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 25s 379ms/step - accuracy: 0.9635 - loss: 0.1245 - val_accuracy: 0.8483 - val_loss: 0.3778
Epoch 7/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 26s 385ms/step - accuracy: 0.9789 - loss: 0.0848 - val_accuracy: 0.8708 - val_loss: 0.3441
Epoch 8/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 25s 380ms/step - accuracy: 0.9827 - loss: 0.0702 - val_accu

In [36]:
y_pred = model.predict(val_ds)

17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 303ms/step


In [37]:
y_pred[0]

array([2.6816533e-08, 3.7294521e-05, 7.6868492e-01, 2.3127776e-01],
      dtype=float32)

In [44]:
for imgs, labels in val_ds.take(1):
    img = imgs[0]
    label = labels[0]

2025-12-10 14:14:27.390908: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [47]:
img_ = Image()

TypeError: 'module' object is not callable